In [57]:
import requests
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt
import klib

pd.set_option("display.max.columns", None)

def get_network_data(asset: str, metrics: list = ['HashRate', 'IssTotUSD', 'FeeTotUSD', 'PriceUSD', 'FeeMeanNtv'], end_time: str = date.today().strftime('%Y-%m-%d')):
    api_str = f"https://community-api.coinmetrics.io/v4/timeseries/asset-metrics?assets={asset}&metrics={','.join(metrics)}&end_time={end_time}&page_size=10000&pretty=true"
    response = requests.get(api_str).json()
    return pd.DataFrame(response['data'])

In [ ]:
df_unclean = get_network_data("ltc", ['HashRate', 'PriceUSD']).fillna("0")

df_cleaned = klib.data_cleaning(df_unclean)

df_cleaned.time = df_cleaned.time.str.replace("T00:00:00.000000000Z", "")
df_cleaned.time = pd.to_datetime(df_cleaned.time)

df_cleaned[['hash_rate', 'price_usd']] =  df_cleaned[['hash_rate', 'price_usd']].apply(pd.to_numeric)

df_cleaned.index = df_cleaned.time
df_cleaned.drop(columns=['time'], inplace=True)

df_cleaned.dtypes
df = df_cleaned
df

In [63]:
# how many days it takes to pay off the machine
# D is the static days-to-breakeven
# cost of histing and machines
# C is the upfront capital expenditure 
# P is the current Litecoin price
# S is hashrate produced by the purchased equipments
# H is the network hashrate
# m is the coinbase reward, currently it is 6.25BTC.
# n is the current avg. transaction fee per block
# k is the efficiency (J/T) of the equipments
# r is the all-in electricity cost ($/KwH)
C = 5_000
P = 150
# Goldshell LT5 Pro MH/s
S = 2050
H = 216 * 1000000
m = 100
n = 1
k = 0.00079
r = 0.1
# Bitmain L7 | 9.5GH@3425W = .002774 rating (360W per GH)
# Goldshell LT5 | 2.05GH@2150W = .000953 rating (1048W per GH)
# Goldshell LT5 Pro | 2.45@3100W = .00079 rating (1265W per GH)
# days to break even mining litecoin on a goldshell LT5 Pro
denominator = P * (S/H) * (m + n) * 6 * 24 - k * S * r * 24
print(C/denominator)

297.29697589516127
